<a href="https://colab.research.google.com/github/vndiaz1/vndiaz1-IIC3633-2020/blob/master/Pr%C3%A1cticos/pyRecLab_SlopeOne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://youtu.be/A2euuevpYis" target="_parent"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/YouTube_full-color_icon_%282017%29.svg/71px-YouTube_full-color_icon_%282017%29.svg.png" alt="Open In Colab"/></a>


# **Práctico Sistemas Recomendadores: pyreclab - Slope One**

En este práctico seguiremos utilizando [pyreclab](https://github.com/gasevi/pyreclab), con el cual estamos aprendiendo distintas técnicas de recomendación. Seguiremos usando la misma base de datos de los prácticos anteriores, para que puedan comparar los métodos y sus implementaciones. Este práctico está acompañado de un [video comentando la actividad](https://youtu.be/A2euuevpYis).

En esta oportunidad exploraremos el recomendador de Pendiente Uno o **Slope One** [1].

**Adaptado y preparado por:** Francisca Cattan 📩 fpcattan@uc.cl

Referencias 📖
------
[1] *Lemire, D., & Maclachlan, A. (2005, April). Slope One Predictors for Online Rating-Based Collaborative Filtering. In SDM (Vol. 5, pp. 1-5).*


**Nombre**:  Vicente Díaz F.


## Actividad 1 👓

Antes de empezar con el práctico, responde la siguiente pregunta con lo visto en clases.

**Pregunta:** Explique cómo funciona Slope One (como modelo teórico, no piense en la implementación). En particular explique:

- Repasemos: ¿Por qué este recomendador es un algoritmo de Filtrado Colaborativo?
- Este Filtrado Colaborativo, ¿está basado en el usuario o en los items? ¿Por qué?
- ¿Qué datos recibe Slope One y qué hace con ellos? (qué tipo de columnas y qué calculo)
- ¿Qué pasaría si se agrega un nuevo rating a la base de datos?
- Opcional: ¿Cómo crees que le iría al recomendador con un usuario que acaba de entrar al sistema y ha asignado muy pocos ratings?

💡 *Hint: La bibliografía todo lo puede.*

**Respuesta:**
- Forma parte del filtrado colaboartivo, ya que predice según los *ratings* que han realizado otros usuarios.
- Se basa en items, porque obtiene los delta entre cada par de items y luego los utiliza para predecir los ratings de un usuario en particular.
- Recibe los *ratings* de cada item y genera un promedio de la diferencia entre cada par de item. Después, lo suma a los *ratings* del usuario activo y se hace un promedio ponderado para obtener la predicción del item que se eligió.
- Al recibir un nuevo *rating* el método lo puede agregar fácilmente al promedio de ese item, pero despúies tendría que recalcular todos las diferencias entre los pares de items que incluyan este item.
- No le iría tan bien, ya que la predicción se basa en el promedio ponderado de las diferencias entre los items que se quiere predecir y los que ha visto ("valorado") el usuario, por lo que si ha visto pocos items, el resultado va a ser poco confiable. Aunque esto es un problrma para la mayoría de los FC.




# **Configuración Inicial**

## Paso 1:
Descargue directamente a Colab los archivos del dataset ejecutando las siguientes 3 celdas:


In [ ]:
!curl -L -o "u1.base" "https://drive.google.com/uc?export=download&id=1bGweNw7NbOHoJz11v6ld7ymLR8MLvBsA"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    490      0 --:--:-- --:--:-- --:--:--   489
100 1546k  100 1546k    0     0  1250k      0  0:00:01  0:00:01 --:--:-- 1250k


In [ ]:
!curl -L -o "u1.test" "https://drive.google.com/uc?export=download&id=1f_HwJWC_1HFzgAjKAWKwkuxgjkhkXrVg"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    627      0 --:--:-- --:--:-- --:--:--   626
100  385k  100  385k    0     0   353k      0  0:00:01  0:00:01 --:--:--  353k


In [ ]:
!curl -L -o "u.item" "https://drive.google.com/uc?export=download&id=10YLhxkO2-M_flQtyo9OYV4nT9IvSESuz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    806      0 --:--:-- --:--:-- --:--:--   806
100  230k  100  230k    0     0   265k      0 --:--:-- --:--:-- --:--:--  265k


Los archivos **u1.base** y **u1.test** tienen tuplas {usuario, item, rating, timestamp}, que es la información de preferencias de usuarios sobre películas en una muestra del dataset [movielens](https://grouplens.org/datasets/movielens/).

## Paso 2:

Instalamos pyreclab utilizando pip.

In [ ]:
!pip install pyreclab --upgrade

     |████████████████████████████████| 194kB 8.1MB/s 


## Paso 3:

Hacemos los imports necesarios para este práctico.

In [ ]:
import pyreclab
import numpy as np
import pandas as pd

# **El dataset**

💡 *En prácticos anteriores, vimos como analizar este dataset. Puedes revisarlos en caso de dudas.*

## Paso 4:

Ya que queremos crear una lista de recomendación de items para un usuario en especifico, necesitamos obtener información adicional de cada película tal como título, fecha de lanzamiento, género, etc. Cargaremos el archivo de items descargado "u.item" para poder mapear cada identificador de ítem al conjunto de datos que lo describe.

In [ ]:
# Definimos el orden de las columnas
info_cols = [ 'movieid', 'title', 'release_date', 'video_release_date', 'IMDb_URL', \
              'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', \
              'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', \
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western' ]

# Asignamos a una variable la estructura de datos de los items
info_file = pd.read_csv('u.item', sep='|', index_col = 0, names = info_cols, header=None, encoding='latin-1')

# **Slope One**

## Paso 5:

Seguiremos un camino muy similar a los ejercicios de User KNN e Item KNN. Crearemos una instancia del algoritmo de recomendación y luego pasaremos a la fase de entrenamiento.

In [ ]:
# Declaramos la instancia SlopeOne
mySlopeOne = pyreclab.SlopeOne(dataset='u1.base', dlmchar=b'\t', header=False, usercol=0, itemcol=1, ratingcol=2)

In [ ]:
# Y enntrenamos
mySlopeOne.train()

## Actividad 2 👓

**Pregunta:** Explique qué hace el método `train()` en este caso, dado el modelo teórico. ¿Calcula información?, ¿no hace nada?, ¿ordena los datos? 

**Respuesta:** El entrenamiento de este método consiste en obtener los promedio de las diferencias entre cada par de items y guardar la cantidad utilizada para el cálculo, para utilizarlo en el promedio ponderado.

## Paso 6:

Llego la hora de predecir el rating.

In [ ]:
# Esta es la predicción de rating que el usuario ID:457 otorgaría al ítem ID:37
# De esta forma podemos comparar el resultado con los prácticos anteriores
mySlopeOne.predict("457", "37")

3.2408759593963623

In [ ]:
# También podemos guardar la predicción en una variable
prediction = mySlopeOne.predict("457", "37")

In [ ]:
# Podemos comprobar las peliculas rankeadas por el usuario ID:457
# Que ciertamente ha participado activamente (¡156 items!)
train_file = pd.read_csv('u1.base', sep='\t', names = ['userid', 'itemid', 'rating', 'timestamp'], header=None)
train_file[train_file['userid'] == 457]

,userid,itemid,rating,timestamp
37269,457,1,4,882393244
37270,457,7,4,882393278
37271,457,9,5,882393485
37272,457,11,4,882397020
37273,457,13,3,882393883
...,...,...,...,...
37420,457,1047,2,882395964
37421,457,1119,4,882398308
37422,457,1168,5,882548761
37423,457,1210,4,882549905


In [ ]:
# Y también cuáles usuarios han rankeado la pelicula ID:37
train_file[train_file['itemid'] == 37]

,userid,itemid,rating,timestamp
1302,13,37,1,882397011
14851,201,37,2,884114635
19670,268,37,3,876514002
29489,363,37,2,891498510
31084,385,37,4,880013483
32996,405,37,1,885548384
62777,773,37,3,888540352


## Actividad 3 👓

Haremos un pequeño experimento para entender mejor como funciona Slope One. Gracias al ejercicio anterior, sabemos que el usuario 457 ya ha asignado el mejor rating (5 ⭐) a las dos peliculas ID:9 e ID:1168. Comparemos.

**Pregunta:** ¿Cómo se explican estos resultados?  

**Respuesta:** Para realizar la predicción, no se toma en cuenta el *rating* del usuario (generalmente no se conoce), por lo que se calcula con el promedio ponderado de las diferencias entre las películas 9 y 1168 con las otras que ha rankeado el usuario 457 para generar el *rating* esperado. Como estos algoritmos son promedios y la idea es que generalicen, no se va a tener un resultado perfecto, pero se puede ver que funciona bastante bien para este caso, ya que si se redondea las valoraciones, se tiene 5 y 4 estrellas, lo que es similar a la opinión del usuario 457.

In [ ]:
prediction_id9 = mySlopeOne.predict("457", "9")
prediction_id1168 = mySlopeOne.predict("457", "1168")

print('Prediction for ID:9 :', prediction_id9)
print('Prediction for ID:1168 :', prediction_id1168)

Prediction for ID:9 : 4.530702114105225
Prediction for ID:1168 : 4.166153907775879


## Paso 7:

Generaremos ahora una lista ordenada de las top-N recomendaciones, dado un usuario.



In [ ]:
# Mediante el método recommend() genereremos una lista top-5 recomendaciones para el usuario ID:457
reclist_slopeone = mySlopeOne.recommend("457", 5)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone)

Lista de items según ID: ['1592', '1589', '1656', '1431', '1653']


In [ ]:
# Lo convertimos a numpy array
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1592                               Magic Hour, The (1998)
1589                                   Schizopolis (1996)
1656                                   Little City (1998)
1431                                  Legal Deceit (1997)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

## Actividad 4 👩🏻‍💻

Genera una nueva recomendacion, modificando los hiperparametros de usuario y topN a tu elección.

**Pregunta:** ¿Ves una diferencia en la recomendación entre el nuevo usuario y el usuario ID:457?

**Respuesta:** Se puede ver que las recomendaciones son totalmente distinas, lo que tiene sentido al ser un filtro personalizado.

In [ ]:
# Escribe el nuevo codigo aqui
reclist_slopeone = mySlopeOne.recommend("300", 8)

# Y visualizaremos el resultado
# print('Lista de items según ID:', reclist_slopeone)
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1064             Crossfire (1947)
1612      Leading Man, The (1996)
1606              Deceiver (1997)
1604    He Walked by Night (1948)
1547             Show, The (1995)
1551     Glass Shield, The (1994)
1450       Golden Earrings (1947)
1536         Aiqing wansui (1994)
Name: title, dtype: object

## Actividad 5 👩🏻‍💻

Dado el usuario ID:44, cree dos listas de películas recomendadas; la primera utilizando el algoritmo Most Popular y la segunda utilizando el algoritmo Slope One.

**Pregunta:** Realice un analisis apreciativo de las similitudes y diferencias entre ambas recomendaciones.

**Respuesta:** Se puede concluir que no hay similitudes entre ambos métodos recomendadores, ya que el top N de películas es completamente distinto. Esto ocurre debido a que el most popular ocupa las películas más vistas, por lo que al predecir sus valoraciones mediante Slope One, estas pueden tener buenas valoraciones (4-4.6) pero no alcanza a las 5 estrellas, ya que es imposible que todas las personas le pongan tan buena valoración. Esto influye porque las que recomienda Slope One son películas que han sido poco valoradas y con valoraciones de 5 estrellas, por lo que es más fácil llegar a predecir con valoración máxima estas películas (se puede ver que todo el top N con Slope One tienen valoraciones 5 estrellas y muy pocos *ratings*.

In [ ]:
# Parameters
USER_ID = "44"
TOP_N   = 10

################ Most popular ################

# Definicion de objeto "most popular"
most_popular = pyreclab.MostPopular(dataset='u1.base',
                   dlmchar=b'\t',
                   header=False,
                   usercol=0,
                   itemcol=1,
                   ratingcol=2)
# en teoria no es "entrenar" , objeto "most_popular" solo registra los items más populares en el set de entrenamiento  
most_popular.train()

reclist_mostpop   = most_popular.recommend(USER_ID, TOP_N)
recmovies_mostpop = np.array(reclist_mostpop).astype(int)
print('Lista de items por nombre:')
info_file.loc[recmovies_mostpop]['title']

Lista de items por nombre:


movieid
50                     Star Wars (1977)
100                        Fargo (1996)
181           Return of the Jedi (1983)
286         English Patient, The (1996)
1                      Toy Story (1995)
121       Independence Day (ID4) (1996)
300                Air Force One (1997)
127               Godfather, The (1972)
7                 Twelve Monkeys (1995)
98     Silence of the Lambs, The (1991)
Name: title, dtype: object

In [ ]:
################ Slope One ################

reclist_slopeone = mySlopeOne.recommend(USER_ID, TOP_N)
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1656                        Little City (1998)
1064                          Crossfire (1947)
1643                         Angel Baby (1995)
1642                  Some Mother's Son (1996)
1625                         Nightwatch (1997)
1599             Someone Else's America (1995)
1512    World of Apu, The (Apur Sansar) (1959)
1536                      Aiqing wansui (1994)
1500                 Santa with Muscles (1996)
1467      Saint of Fort Washington, The (1993)
Name: title, dtype: object

In [ ]:
# Predicciones
MOVIE_ID = "50"
pred_so = mySlopeOne.predict(USER_ID, MOVIE_ID)

print(f"Predicciones de la película {MOVIE_ID}")
print(f"Slope One: {pred_so}")

Predicciones de la película 50
Slope One: 4.506339073181152


In [ ]:
# train_file[train_file['userid'] == int(USER_ID)]
# print()
train_file[train_file['itemid'] == int(MOVIE_ID)]


,userid,itemid,rating,timestamp
36,1,50,5,874965954
220,2,50,5,888552084
298,4,50,5,892003526
615,7,50,5,891351042
914,8,50,5,879362124
...,...,...,...,...
79430,937,50,5,876769374
79472,938,50,5,891356314
79631,940,50,4,885921542
79754,942,50,5,891282816
